In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
pd.core.common.is_list_like = pd.api.types.is_list_like
import fix_yahoo_finance as yf
from pandas_datareader import data as pdr
from scipy.stats import norm

In [2]:
ticker = ['SAN.MC','TEF.MC','SGRE.MC','ENG.MC','ITX.MC'] # TAKE THESE 5 STOCKS FROM IBEX 35 INDEX AS A EXAMPLE
weights = np.array([0.2,0.2,0.2,0.2,0.2]) #CREATE AN ARRAY WITH THE WEIGHTS
fecha_inicio = '2017-01-01'
fecha_final = dt.datetime.today()

In [3]:
yf.pdr_override() # GET THE DATA FROM YAHOO FINANCE
security = pdr.get_data_yahoo(ticker, 
                           fecha_inicio,
                            fecha_final)['Adj Close']

[*********************100%***********************]  5 of 5 downloaded


In [4]:
security.head()

,ENG.MC,ITX.MC,SAN.MC,SGRE.MC,TEF.MC
Date,,,,,
2017-01-02,22.272337,31.342033,4.478954,17.511244,8.054496
2017-01-03,21.912737,31.029575,4.587560,17.466572,8.132766
2017-01-04,21.703350,30.736343,4.608213,17.404032,8.148237
2017-01-05,21.817148,30.236410,4.621674,17.323624,8.321159
2017-01-06,21.780733,30.452726,4.647701,17.386164,8.368484


In [5]:
returns = security.pct_change()
returns.head()

,ENG.MC,ITX.MC,SAN.MC,SGRE.MC,TEF.MC
Date,,,,,
2017-01-02,NaN,NaN,NaN,NaN,NaN
2017-01-03,-0.016146,-0.009969,0.024248,-0.002551,0.009718
2017-01-04,-0.009555,-0.009450,0.004502,-0.003581,0.001902
2017-01-05,0.005243,-0.016265,0.002921,-0.004620,0.021222
2017-01-06,-0.001669,0.007154,0.005632,0.003610,0.005687


In [6]:
#COVARIANCE MATRIX
cov_matrix = returns.cov()
cov_matrix

,ENG.MC,ITX.MC,SAN.MC,SGRE.MC,TEF.MC
ENG.MC,0.000095,0.000036,0.000040,0.000032,0.000049
ITX.MC,0.000036,0.000142,0.000052,0.000051,0.000049
SAN.MC,0.000040,0.000052,0.000189,0.000065,0.000091
SGRE.MC,0.000032,0.000051,0.000065,0.000450,0.000045
TEF.MC,0.000049,0.000049,0.000091,0.000045,0.000113


In [7]:
avg_rets = returns.mean()
avg_rets

ENG.MC     0.000243
ITX.MC    -0.000440
SAN.MC    -0.000032
SGRE.MC   -0.000552
TEF.MC    -0.000300
dtype: float64

In [8]:
port_ret = avg_rets.dot(weights) #DAILY

In [9]:
port_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) #DAILY

In [10]:
investment = 100000

In [11]:
mean_investment = (1 + port_ret) * investment

In [12]:
investmente_stv = investment * port_std

In [13]:
conf_levl = 0.01
print('we are calculating var ' +str((1- conf_levl)*100)+' %')

we are calculating var 99.0 %


# calculating the var

In [14]:
cut = norm.ppf(conf_levl, mean_investment,investmente_stv)
cut

97892.18574604872

In [15]:
var_1d = investment - cut
var_1d
print('In one day you can lose : '+str(np.round(var_1d,2)) +' euros')
print('this is equivalent to: ' +str (np.round((var_1d / investment)*100,2))+' % of your portfolio')

In one day you can lose : 2107.81 euros
this is equivalent to: 2.11 % of your portfolio


In [16]:
# Var at different time frames

In [17]:
num_days = 8

In [18]:
print('ANNUAL VAR: ' +str(np.round(var_1d * np.sqrt(252),2))+ ' EUROS')

ANNUAL VAR: 33460.51 EUROS


In [19]:
for x in range(0,num_days):
    print(str(x) + ' day VaR: ' + str(np.round(var_1d * np.sqrt(x), 2))) 

0 day VaR: 0.0
1 day VaR: 2107.81
2 day VaR: 2980.9
3 day VaR: 3650.84
4 day VaR: 4215.63
5 day VaR: 4713.22
6 day VaR: 5163.07
7 day VaR: 5576.75
